In [1]:
%config Completer.use_jedi = False

In [2]:
reset -fs

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from sklearn.neighbors       import KNeighborsClassifier

from sklearn.naive_bayes     import GaussianNB

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm             import SVC

from sklearn.decomposition   import PCA

import imblearn
from   imblearn.pipeline          import make_pipeline # scikit-learn Pipeline does not work with imblearn

from sklearn.metrics import balanced_accuracy_score, fbeta_score, f1_score


from imblearn.pipeline import Pipeline

# Data Science Research Question
-----
## Can we develop an ML model to predict whether or not a patient will have a death event based on common heart failure predictors?

In [6]:
data = pd.read_csv('data/heart_failure_clinical_records_dataset.csv')
data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


First after very basic EDA, I summarize the features in this dataset, and how they relate to heart failure:
- **Age** - age of patient in years
- **Anaemia** - whether or not a patient has a decrease of red blood cells (boolean)
- **Creatinine Phosphokinase** - level of CPK enzyme in blood in (high values may indicate muscle damage - heart is a muscle)
- **Diabetes** - whether or not a patient has diabetes (boolean)
- **Ejection Fraction** - percentage of blood being pumped out of left ventricle (lower values may indicate issues)
- **High Blood Pressure** - whether or not a patient has hypertension (boolean)
- **Platelets** - concentration of platelets in blood in kiloplatelets/mL
- **Serum Creatinine** - concentration of creatinine in blood in mg/dL (increased levels are a marker of poor cardiac output).
- **Serum Sodium** - level of sodium in blood in mEq/L (low concentration is a biological marker for heart failure)
- **Sex** - whether the patient is male or female (binary: 1=Male, 0=Female)
- **Smoking** - if the patient smokes (boolean)
- **Time** - the follow up period for the patient up to the death event (whether they died or were censored).
- **Death Event** - (target), whether or not the patient died during the follow up (boolean)
    - 1 - patient died
    - 0 - patient was censored (scientist lost contact with patient) 

Since the time variable is populated when the scientists either lost contact with the patient, or the patient died, it is indicative of survival, and obviously not known beforehand. In other words, if we we're to deploy a working model with all the columns and someone wanted to predict whether a certain patient was likely to have a death event, they would have no "time" variable to put as an input (since that is recorded when the patient dies). Therefore, I made the decision initially to drop time as a predictor.

In [7]:
y = data.DEATH_EVENT # Our target variable is DEATH_EVENT 
X = data.drop(columns=['DEATH_EVENT', 'time'])  # Remove the target variable and time from X

In [8]:
# How many observations do we have?
len(X)

299

After importing the data, the next step is to pull off a segment of the data that will be our testing set. This test set will be be hidden away from the models we are designing until we are ready to test one. Additionally, since there are only have 299 observations, I am going to split the data by a slightly higher percentage (25% as opposed to 20%) to ensure that the testing set is somewhat representative of the data.

In [9]:
# Testing split that we are going to hide away from our model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25) 

# Feature Engineering
-----

After performing more extensive EDA (in a different notebook), I noticed that all of the columns were numeric types, but some represented categorical variables (i.e. whether or not someone smokes, has high blood pressure, etc.). In the dataset, these columns are already one-hot-encoded with binary 1s or 0s to indicate whether someone exhibits that characteristic. Therefore, in my feature engineering one-hot-encoding is not needed. Additionally, there are no nan values in the training data, however I still impute for missing values, as they might be included in the testing set or data we want to use the model on.

For my feature engineering, the steps I took were:
1. Identify categorical and numerical columns and establish pipelines for each


2. Scale the data
    - For my numeric data I chose to use StandardScaler(), which normalizes columns to fall in line with the normal distribution. This is important as many algorithms may perform poorly if individual features do not look more or less like a standard normally distributed data.
    - It doesn't make sense to use a scaler on my categorical columns since they're simply one-hot-encoded as 1s or 0s.


3. Impute missing values with SimpleImputer()
    - For numerical columns I used the strategy "median" whereas for categorical I used "most_frequent". These methods fill any missing values with the corresponding strategy applied to the column.

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
dtypes: float64(3), int64(8)
memory usage: 25.8 KB


In [14]:
cat_cols = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']  # These are all binary
con_cols = ['creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'age']

In [15]:
con_pipe = Pipeline([('scaler', StandardScaler()),  # Standard scaler for numerical variables
                     ('imputer', SimpleImputer(strategy='median', add_indicator=True))])


cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent'))])  # Shouldn't standardize binary variables



# Apply numerical and categorical pipeline to pre-processing step
preprocessing = ColumnTransformer([('categorical', cat_pipe, cat_cols), 
                                   ('continuous', con_pipe, con_cols)])

In [102]:
# con_pipe = make_pipeline(imblearn.over_sampling.SMOTE(), StandardScaler(), SimpleImputer(strategy='median'))

# age_pipe = make_pipeline(imblearn.over_sampling.SMOTE(), MinMaxScaler((0,1)), SimpleImputer(strategy='median'))

# cat_pipe = make_pipeline(imblearn.over_sampling.SMOTE(), SimpleImputer(strategy='most_frequent'))

# preprocessing = ColumnTransformer([('categorical', cat_pipe, cat_cols),
#                                    ('continuous', con_pipe, con_cols),
#                                    ('age', age_pipe, age_col)])

Additionally, when looking at the target variable Death_event, I saw evidence of imbalanced data as the percentage of death events in the target variable was around 32%. Therefore, in my pipeline for the randomized search CV, I will 

In [24]:
np.unique(y, return_counts=True)

(array([0, 1]), array([203,  96]))

# Algorithms & Search
-----

To analyze which algorithm has the best performance I utilize automated algorithm and hyperparameter search and used f1 as my evaluation metric (since it works well for classification). When picking algorithms, I focused on a variety of ones that would be good for binary classification. The three algorithms I chose to compare were

1. `LogisticRegression()` - I chose this model because it is a linear model for *classification*, which is the scope of my research question in this case.
    - Important parameters:
     - `penalty` - Specifies the normalization used in the penalty term
     
     - `C` - Inverse of regularization strength was varied (smaller values result in stronger regularization)
     
     - `solver` - Algorithm that is used in hte optimization problem 
     
     - `class_weight` - The weights associated with specific classes (if we have evidence of imbalanced data, setting to `balanced` may be helpful
     
     
2. `RandomForestClassifier()` - I chose this model because fitting a number of decision tree classifiers on various sub-samples of the data will improve accuracy in predictions and reduce overfitting 
    - Important parameters:
        - `n_estimators` - number of trees in the forest, important because adding more trees trained on different subsets of the data reduces variance
        - `criterion` - function used to determined the effectiveness of a split
        - `bootstrap` - whether or not to use bootstrap samples
            - **bootstrapping** is introducing amnesia by training trees on only a portion of the data, weakens the trees to improve *generality*
        - `min

In [16]:
class DummyEstimator(BaseEstimator):
    "Pass through class, methods are present but do nothing."
    def fit(self): pass
    def score(self): pass

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
np.linspace(0.01, 5, 10)

array([0.01      , 0.56444444, 1.11888889, 1.67333333, 2.22777778,
       2.78222222, 3.33666667, 3.89111111, 4.44555556, 5.        ])

In [22]:

# pipe = make_pipeline(imblearn.over_sampling.SMOTE(),
#                      preprocessing,
#                      PCA(),
#                      DummyEstimator()
#                      )

pipe = Pipeline(steps = [('smote', imblearn.over_sampling.SMOTE()),
                         ('preprocessing', preprocessing),
                         ('clf', DummyEstimator())])

# pipe = Pipeline(steps=[()])


search_space = [
    {'clf': [LogisticRegression(n_jobs=-1)],
        'clf__C': np.linspace(0.01, 5, 10),
        'clf__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'clf__class_weight': ['balanced', None],
        'clf__penalty': ['l1', 'l2', 'elasticnet', 'none']},
    
    {'clf': [RandomForestClassifier(n_jobs=-1)],
        'clf__criterion': ['gini', 'entropy'],
        'clf__min_samples_leaf': np.linspace(1, 10, 6, dtype=int),
        'clf__bootstrap': [True, False],
        'clf__class_weight': [None, 'balanced', 'balanced_subsample'],
        'clf__n_estimators': np.linspace(0, 200, 100, dtype=int)},
    
    {'clf': [SVC()], 
         'clf__class_weight': ['balanced', None],
         'clf__C': np.linspace(1, 100, 10),
         'clf__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
         'clf__degree': range(1,10)}, 
    
    {'clf': [KNeighborsClassifier(n_jobs=-1)],
         'clf__leaf_size': np.linspace(5, 50, 5, dtype=int),
         'clf__n_neighbors': np.linspace(3, 13, 4, dtype=int),
         'clf__weights': ['uniform', 'distance'],
         'clf__p': [1,2]}, 
    
    {'clf': [GaussianNB()]}, 
    
    {'clf': [ExtraTreesClassifier(n_jobs=-1)], 
         'clf__criterion': ['gini', 'entropy'],
         'clf__min_samples_leaf': np.linspace(1, 30, 5, dtype=int),
         'clf__bootstrap': [True, False],
         'clf__class_weight': [None, 'balanced', 'balanced_subsample'],
         'clf__n_estimators': np.linspace(0, 500, 100, dtype=int)}
]


In [23]:
gs = RandomizedSearchCV(pipe, 
                        search_space, 
                        scoring='f1_weighted', 
                        n_iter=30,
                        cv=5,
                        n_jobs=-1
                        )

gs.fit(X_train, y_train)

best_model = gs.best_params_['clf']

gs.best_score_, gs.best_params_

(0.7760376301744614,
 {'clf__n_estimators': 6,
  'clf__min_samples_leaf': 10,
  'clf__criterion': 'entropy',
  'clf__class_weight': 'balanced_subsample',
  'clf__bootstrap': False,
  'clf': RandomForestClassifier(bootstrap=False, class_weight='balanced_subsample',
                         criterion='entropy', min_samples_leaf=10, n_estimators=6,
                         n_jobs=-1)})

In [126]:
grid = {"n_estimators": np.arange(0,200,2)}
rf = RandomForestClassifier()
rf_random = GridSearchCV(rf, grid, cv=3)
rf_random.fit(X_train, y_train)

/Users/kylebrooks/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kylebrooks/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kylebrooks/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 348, in fit
    self._validate_estimator()
  File "/Users/kylebrooks/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/ensemble/_base.py", line 134, in _validate_estimator
    raise ValueError("n_estimators must be greater than zero, "
ValueError: n_estimators must be greater than zero, got 0.

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/kylebrooks/opt/anaconda3/envs/ml/lib/

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': array([  0,   2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,
        26,  28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,
        52,  54,  56,  58,  60,  62,  64,  66,  68,  70,  72,  74,  76,
        78,  80,  82,  84,  86,  88,  90,  92,  94,  96,  98, 100, 102,
       104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128,
       130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154,
       156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180,
       182, 184, 186, 188, 190, 192, 194, 196, 198])})

In [128]:
print(rf_random.best_params_)
print(rf_random.best_estimator_)

{'n_estimators': 122}
RandomForestClassifier(n_estimators=122)


In [49]:
# sorted(sklearn.metrics.SCORERS.keys())

In [51]:
score_list = []
for i in range(10):
    gs = RandomizedSearchCV(pipe, 
                        search_space, 
                        scoring='f1', 
                        n_iter=30,
                        cv=5,
                        n_jobs=-1
                        )

    gs.fit(X_train, y_train)

    score_list.append(gs.best_score_)

with_smote = np.mean(score_list)

with_smote

0.5180524614688842

In [55]:
score_list = []
for i in range(10):
    gs = RandomizedSearchCV(pipe, 
                        search_space, 
                        scoring='f1', 
                        n_iter=30,
                        cv=5,
                        n_jobs=-1
                        )

    gs.fit(X_train, y_train)

    score_list.append(gs.best_score_)

without_smote = np.mean(score_list)

without_smote

0.519672000277809

In [46]:
gs = RandomizedSearchCV(pipe, 
                        search_space, 
                        scoring='f1', 
                        n_iter=30,
                        cv=5,
                        n_jobs=-1
                        )

gs.fit(X_train, y_train)

gs.best_score_, gs.best_params_

(0.5244444444444445,
 {'clf__n_estimators': 50,
  'clf__min_samples_leaf': 4,
  'clf__criterion': 'gini',
  'clf__class_weight': None,
  'clf__bootstrap': True,
  'clf': RandomForestClassifier(min_samples_leaf=4, n_estimators=50, n_jobs=-1)})

In [56]:
imb_list = [imblearn.over_sampling.SMOTE(), imblearn.over_sampling.ADASYN(), imblearn.over_sampling.BorderlineSMOTE(), imblearn.over_sampling.SVMSMOTE(),
            imblearn.combine.SMOTEENN(), imblearn.combine.SMOTETomek()]

In [58]:
for imb in imb_list:
    total = 0
    for i in range(10):
        pipe = Pipeline(steps = [('imb', imb),
                         ('preprocessing', preprocessing),
                         ('clf', DummyEstimator())])

        gs = RandomizedSearchCV(pipe, 
                        search_space, 
                        scoring='f1', 
                        n_iter=30,
                        cv=5,
                        n_jobs=-1
                        )

        gs.fit(X_train, y_train)

        total += gs.best_score_
        
    avg = total / 10
    
    print(f'{imb} - {avg}')

SMOTE() - 0.5308903680015374
ADASYN() - 0.5362001898682767
BorderlineSMOTE() - 0.5145619642502345
SVMSMOTE() - 0.5105900477783913


/Users/kylebrooks/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1351: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn("'n_jobs' > 1 does not have any effect when"


SMOTEENN() - 0.48616156601531335
SMOTETomek() - 0.5305956924570111


In [222]:
model = gs.best_params_['dummyestimator']
model

RandomForestClassifier(class_weight='balanced_subsample', min_samples_leaf=10,
                       n_estimators=300, n_jobs=-1)

# Evaluation Metrics
----

Now that I have our ideal model based on automated hyperparameter search and model selection, next I look at a variety of evaluation metrics on the training data to assess our models performance on the training set. The first metric I looked at was the weighted f1 score since that is what I used for the cross validation.

In [224]:
pipe = Pipeline(steps = [('preprocessing', preprocessing), 
                         ('clf', model)])

In [226]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_train)
f1_score(y_train, y_pred, average='weighted')

0.8991584936439413

In [230]:
f1_score(y_train, y_pred, average='weighted')

0.8991584936439413

I chose to look at a fbeta_score metric with a beta=2. I chose this metric since in a business setting use of this ML model deals with human lives. Therefore, this model should focus on reducing the amount of false negatives (we would rather tell someone they are at risk of a death event when they're not than miss identifying a person who has a death event. For this reason, I chose to look at fbeta_score with a beta value greater than 1 (as this puts more emphasis on false positives).

In [232]:
fbeta_score(y_train, y_pred, beta=2)

0.8793103448275862

Another evaluation metric I chose to look at 